In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
import time

datapath = '../data-unversions/p1ch7/'
cifar10 = datasets.CIFAR10(root= datapath, train=True, download = True, transform=transforms.ToTensor())

Files already downloaded and verified


# **Pre-processing and Training**


> **Separating desired Features and Outputs**


In [ ]:
imgs = torch.stack([img_t for img_t, _ in cifar10], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

normalize = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean,std)])

train_data = datasets.CIFAR10(root= datapath, train=True, download = True, transform=normalize)
cifar10_train = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

valid_data = datasets.CIFAR10(root= datapath, train=False, download = True, transform=normalize)
cifar10_valid = torch.utils.data.DataLoader(valid_data, batch_size=64, shuffle=True)


Files already downloaded and verified
Files already downloaded and verified



> **Training and Validation Loop**


In [ ]:
def training(epochs, optimizer, model, loss_fn, train_set, valid_set):
  trainStart = time.time()
  for epoch in range(epochs+1):
    for imgs, labels in train_set:
      batch = imgs.shape[0]
      outputs = model(imgs.view(batch, -1))
      train_loss = loss_fn(outputs, labels)

      optimizer.zero_grad()
      train_loss.backward()
      optimizer.step()

    if epoch % 50 == 0:
      print("Epoch: %d, Training Loss: %f" % (epoch, float(train_loss)))
  trainStop = time.time()
  trainDuration = trainStop - trainStart

  print(f'Total Training Time: {trainDuration} seconds')

### Validation Evaluation ###

  total = 0
  correctClass = 0
  validStart = time.time()
  with torch.no_grad():
    for imgs, labels in valid_set:
      batch = imgs.shape[0]
      outputs = model(imgs.view(batch, -1))
      _, predicted = torch.max(outputs, dim=1)
      total += labels.shape[0]
      correctClass += int((predicted == labels).sum())

  validStop = time.time()
  validDuration = validStop - validStart

  print(f'\nValidation Accuracy: {(correctClass/total) * 100}%')
  print(f'Total Validation Time: {validDuration} seconds')
  print(f'Total Runtime: {trainDuration + validDuration} seconds')

**2a. SGD Optimized Neural Network with 1 hidden layer**

In [ ]:
seq_model = nn.Sequential(
    nn.Linear(3072, 512),
    nn.ReLU(),
    nn.Linear(512, 10))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

training(
    epochs = 300,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.CrossEntropyLoss(),
    train_set = cifar10_train,
    valid_set = cifar10_valid
)

Epoch: 0, Training Loss: 2.135784
Epoch: 50, Training Loss: 0.913879
Epoch: 100, Training Loss: 1.141922
Epoch: 150, Training Loss: 0.869871
Epoch: 200, Training Loss: 0.537757
Epoch: 250, Training Loss: 0.237564
Epoch: 300, Training Loss: 0.276458
Total Training Time: 4966.341771602631 seconds

Validation Accuracy: %f 0.5343
Total Validation Time: 2.4173367023468018 seconds
Total Runtime: 4968.759108304977 seconds


**2b. SGD Optimized Neural Network with 3 hidden layer**

In [ ]:
seq_model = nn.Sequential(
    nn.Linear(3072, 512),
    nn.ReLU(),
    nn.Linear(512, 1024),
    nn.ReLU(),
    nn.Linear(1024, 256),
    nn.ReLU(),
    nn.Linear(256,10))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

training(
    epochs = 300,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.CrossEntropyLoss(),
    train_set = cifar10_train,
    valid_set = cifar10_valid
)

Epoch: 0, Training Loss: 2.248029
Epoch: 50, Training Loss: 1.029711
Epoch: 100, Training Loss: 0.401296
Epoch: 150, Training Loss: 0.141769
Epoch: 200, Training Loss: 0.107162
Epoch: 250, Training Loss: 0.009748
Epoch: 300, Training Loss: 0.011992
Total Training Time: 6364.693591594696 seconds

Validation Accuracy: %f 0.5169
Total Validation Time: 2.7211508750915527 seconds
Total Runtime: 6367.414742469788 seconds
